In [2]:
answer

NameError: name 'answer' is not defined

In [ ]:
from collections import deque

def solve():
    # Read the grid from stdin
    with open('input20.txt') as f:
        grid = [line.strip() for line in f]
    n = len(grid)
    m = len(grid[0])
    
    # Find S and E
    start = None
    end = None
    for i in range(n):
        for j in range(m):
            if grid[i][j] == 'S':
                start = (i, j)
            elif grid[i][j] == 'E':
                end = (i, j)
                
    # Directions for movement
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    
    def in_bounds(x,y):
        return 0 <= x < n and 0 <= y < m
    
    # Check if cell is track (including S,E, or '.')
    def is_track(x,y):
        return grid[x][y] in ('.','S','E')
    
    # BFS function to get shortest dist from a single source to all track cells
    def bfs(start_positions):
        dist = [[float('inf')]*m for _ in range(n)]
        q = deque()
        for sx, sy in start_positions:
            dist[sx][sy] = 0
            q.append((sx, sy))
        while q:
            x, y = q.popleft()
            for dx, dy in directions:
                nx, ny = x+dx, y+dy
                if in_bounds(nx, ny) and is_track(nx, ny):
                    if dist[nx][ny] > dist[x][y] + 1:
                        dist[nx][ny] = dist[x][y] + 1
                        q.append((nx, ny))
        return dist
    
    # Get shortest distance from S to all track cells
    dist_from_S = bfs([start])
    # Get shortest distance to E (reverse BFS from E)
    dist_to_E = bfs([end])
    
    baseline_time = dist_from_S[end[0]][end[1]]
    
    # If E is unreachable even normally, then no cheats matter
    if baseline_time == float('inf'):
        # No cheats can save any time
        print(0)
        return
    
    # We want to consider all track cells for starting a cheat (A)
    # and all track cells reachable by 1 or 2 moves ignoring walls for B.
    # We'll enumerate cheats.

    # To avoid duplicates, store cheats in a set: ((Ax,Ay),(Bx,By))
    found_cheats = {}
    
    # Precompute all track cells
    track_cells = [(x,y) for x in range(n) for y in range(m) if is_track(x,y)]
    track_set = set(track_cells)
    
    # For each track cell A, consider B cells:
    # B can be a neighbor (1 step cheat)
    # or a cell reachable by 2 steps ignoring walls (2 step cheat)
    # Moves allowed through anything (# or .), but B must be track.
    
    # Let's define a helper to get possible B cells:
    def possible_cheat_ends(x, y):
        results = []
        # 1-step cheat:
        for dx, dy in directions:
            nx, ny = x+dx, y+dy
            if in_bounds(nx, ny) and is_track(nx, ny):
                # can end here in 1 step through walls if needed
                results.append((nx, ny))
            elif in_bounds(nx, ny):
                # If it's a wall, we can still pass since cheat ignores walls,
                # but we need to end on track. 1 step cheat must end on track,
                # so if this is a wall, can't end here.
                pass
        
        # 2-step cheat:
        # For each intermediate step (even if wall), and final must be track.
        for dx1, dy1 in directions:
            ix, iy = x+dx1, y+dy1
            if in_bounds(ix, iy): # intermediate cell could be anything
                for dx2, dy2 in directions:
                    fx, fy = ix+dx2, iy+dy2
                    if in_bounds(fx, fy) and (fx, fy) in track_set:
                        # This is a valid 2-step cheat end
                        results.append((fx, fy))
        
        return results

    cheat_count = 0
    
    # We'll store savings in a dictionary by cheat start/end to handle duplicates.
    cheat_savings_map = {}
    
    for (Ax, Ay) in track_cells:
        if dist_from_S[Ax][Ay] == float('inf'):
            # If we can't reach this track cell from S normally, no point.
            continue
        possible_ends = possible_cheat_ends(Ax, Ay)
        for (Bx, By) in possible_ends:
            if dist_to_E[Bx][By] == float('inf'):
                # Can't reach E from this cell, skip
                continue
            # Time with cheat:
            time_with_cheat = dist_from_S[Ax][Ay] + 2 + dist_to_E[Bx][By]
            saving = baseline_time - time_with_cheat
            if saving > 0:
                # Identify the cheat uniquely by ((Ax,Ay),(Bx,By))
                cheat_key = ((Ax, Ay), (Bx, By))
                # Only record the best saving if multiple paths give same cheat
                if cheat_key not in cheat_savings_map or cheat_savings_map[cheat_key] < saving:
                    cheat_savings_map[cheat_key] = saving
    
    # Now count how many have saving >= 100
    answer = sum(1 for s in cheat_savings_map.values() if s >= 100)
    print(answer)

solve()


1176


In [8]:
from collections import deque

def solve():
    with open('input20.txt') as f:
        grid = [line.strip() for line in f]
    n = len(grid)
    m = len(grid[0])
    
    # Find S and E
    start = None
    end = None
    for i in range(n):
        for j in range(m):
            if grid[i][j] == 'S':
                start = (i, j)
            elif grid[i][j] == 'E':
                end = (i, j)
                
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    def in_bounds(x,y):
        return 0 <= x < n and 0 <= y < m
    def is_track(x,y):
        return grid[x][y] in ('.','S','E')
    
    # BFS for shortest distance from S
    def bfs_starts(starts):
        dist = [[float('inf')] * m for _ in range(n)]
        q = deque()
        for (sx, sy) in starts:
            dist[sx][sy] = 0
            q.append((sx, sy))
        while q:
            x,y = q.popleft()
            for dx, dy in directions:
                nx, ny = x+dx, y+dy
                if in_bounds(nx, ny) and is_track(nx, ny) and dist[nx][ny] > dist[x][y] + 1:
                    dist[nx][ny] = dist[x][y] + 1
                    q.append((nx, ny))
        return dist
    
    dist_from_S = bfs_starts([start])
    dist_to_E = bfs_starts([end])
    baseline_time = dist_from_S[end[0]][end[1]]
    if baseline_time == float('inf'):
        # E not reachable normally
        print(0)
        return
    
    track_cells = [(x,y) for x in range(n) for y in range(m) if is_track(x,y)]
    track_set = set(track_cells)
    
    cheat_savings_map = {}
    
    # For each track cell A, run a BFS ignoring walls, up to 20 steps
    for Ax, Ay in track_cells:
        if dist_from_S[Ax][Ay] == float('inf'):
            continue
        
        # BFS ignoring walls from A for up to 20 steps
        # We'll store the minimum steps to reach each cell (including walls)
        dist_cheat = [[float('inf')] * m for _ in range(n)]
        dist_cheat[Ax][Ay] = 0
        q = deque()
        q.append((Ax, Ay))
        
        while q:
            x, y = q.popleft()
            current_steps = dist_cheat[x][y]
            if current_steps == 20:
                # Can't go further than 20 steps
                continue
            for dx, dy in directions:
                nx, ny = x+dx, y+dy
                if in_bounds(nx, ny):
                    # Can pass through walls
                    next_cost = current_steps + 1
                    if next_cost < dist_cheat[nx][ny]:
                        dist_cheat[nx][ny] = next_cost
                        q.append((nx, ny))
        
        # Now dist_cheat gives the minimum steps needed to reach every cell in <=20 steps ignoring walls
        # Only consider those ending on track
        for Bx, By in track_cells:
            steps_needed = dist_cheat[Bx][By]
            if steps_needed == float('inf') or steps_needed == 0:
                continue
            # We have a potential cheat from A to B with cost steps_needed
            if dist_to_E[Bx][By] == float('inf'):
                # Can't reach E from B, skip
                continue
            # Time with cheat:
            time_with_cheat = dist_from_S[Ax][Ay] + steps_needed + dist_to_E[Bx][By]
            saving = baseline_time - time_with_cheat
            if saving > 0:
                cheat_key = ((Ax, Ay), (Bx, By))
                if cheat_key not in cheat_savings_map or cheat_savings_map[cheat_key] < saving:
                    cheat_savings_map[cheat_key] = saving
    
    # Count how many have saving >= 100
    answer = sum(1 for s in cheat_savings_map.values() if s >= 100)
    print(answer)

solve()

982891
